In [7]:
import basd

import os
import rioxarray as rio
from rasterio.enums import Resampling
import xarray as xr

In [6]:
os.listdir('../basd/data')

['hurs_obs-hist_coarse_1979-2014.nc',
 'hurs_obs-hist_fine_1979-2014.nc',
 'hurs_sim-fut-basd_coarse_2065-2100.nc',
 'hurs_sim-fut_coarse_2065-2100.nc',
 'hurs_sim-hist_coarse_1979-2014.nc',
 'prsnratio_obs-hist_coarse_1979-2014.nc',
 'prsnratio_obs-hist_fine_1979-2014.nc',
 'prsnratio_sim-fut-basd_coarse_2065-2100.nc',
 'prsnratio_sim-fut-basd_fine_2065-2100.nc',
 'prsnratio_sim-fut_coarse_2065-2100.nc',
 'prsnratio_sim-hist_coarse_1979-2014.nc',
 'pr_obs-hist_coarse_1979-2014.nc',
 'pr_obs-hist_fine_1979-2014.nc',
 'pr_sim-fut-basd_coarse_2065-2100.nc',
 'pr_sim-fut-basd_fine_2065-2100.nc',
 'pr_sim-fut_coarse_2065-2100.nc',
 'pr_sim-hist_coarse_1979-2014.nc',
 'ps_obs-hist_coarse_1979-2014.nc',
 'ps_obs-hist_fine_1979-2014.nc',
 'ps_sim-fut-basd_coarse_2065-2100.nc',
 'ps_sim-fut_coarse_2065-2100.nc',
 'ps_sim-hist_coarse_1979-2014.nc',
 'rlds_obs-hist_coarse_1979-2014.nc',
 'rlds_obs-hist_fine_1979-2014.nc',
 'rlds_sim-fut-basd_coarse_2065-2100.nc',
 'rlds_sim-fut-basd_fine_2065-21

In [34]:
data_path = '../basd/data'
obs_hist_path = 'pr_obs-hist_fine_1979-2014.nc'
sim_hist_path = 'pr_sim-hist_coarse_1979-2014.nc'
sim_fut_path = 'pr_sim-fut_coarse_2065-2100.nc'
pr_obs_hist = xr.open_dataset(os.path.join(data_path, obs_hist_path),
                              decode_coords = 'all')
pr_sim_hist = xr.open_dataset(os.path.join(data_path, sim_hist_path),
                              decode_coords = 'all')
pr_sim_fut = xr.open_dataset(os.path.join(data_path, sim_fut_path),
                             decode_coords = 'all')

In [38]:
# Coordinate system definition
pr_sim_hist.rio.write_crs(4326, inplace=True)
pr_obs_hist.rio.write_crs(4326, inplace=True)
pr_sim_fut.rio.write_crs(4326, inplace=True)

<xarray.Dataset>
Dimensions:      (lat: 2, lon: 2, time: 13148)
Coordinates:
  * lat          (lat) float64 53.5 52.5
  * lon          (lon) float64 12.5 13.5
  * time         (time) datetime64[ns] 2065-01-01 2065-01-02 ... 2100-12-31
    spatial_ref  int32 0
Data variables:
    pr           (lon, lat, time) float32 ...

In [41]:
# temporarily renaming (lon, lat) --> (x, y)
# and ordering dimensions as time, y, x
sim_hist_xy = pr_sim_hist.rename({'lon': 'x', 'lat': 'y'}).transpose('time', 'y', 'x')
obs_hist_xy = pr_obs_hist.rename({'lon': 'x', 'lat': 'y'}).transpose('time', 'y', 'x')
sim_fut_xy = pr_sim_fut.rename({'lon': 'x', 'lat': 'y'}).transpose('time', 'y', 'x')

In [42]:
obs_hist_resized_xy = obs_hist_xy.rio.reproject_match(sim_hist_xy,
                                                      resampling = Resampling.bilinear)
obs_hist_resized_lat_lon = obs_hist_resized_xy.rename({'x': 'lon', 'y': 'lat'})

In [43]:
obs_hist_resized_lat_lon

<xarray.Dataset>
Dimensions:      (lon: 2, lat: 2, time: 13149)
Coordinates:
  * lon          (lon) float64 12.5 13.5
  * lat          (lat) float64 53.5 52.5
  * time         (time) datetime64[ns] 1979-01-01 1979-01-02 ... 2014-12-31
    spatial_ref  int32 0
Data variables:
    pr           (time, lat, lon) float32 1.279e-05 1.872e-05 ... 6.653e-06